In [1]:
import pandas as pd
import numpy as np
from utils import utils

In [ ]:
### Y-target is SEVERITY_CLASS, steps followed:
### 1. training the model, 2. using Cross validation and HPT, 3. getting metrics, 4. checking the model validated model on holdout set

In [2]:
df=pd.read_csv('final_ohc_wto_sde_train.csv')
df

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20080506X00599,1,0.012294,0.005339,0.086957,0.00,0.000000,0.25,0.324324,0.225806,...,0,0,0,1,0,0,0,1,1,0
1,20090701X43709,1,0.016400,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.129032,...,0,1,0,0,0,1,0,0,1,0
2,20080827X01344,2,0.014951,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,0,1
3,20090811X65603,1,0.021714,0.005339,0.142068,0.03,0.000000,0.25,0.783784,0.258065,...,0,0,1,0,0,1,0,0,1,0
4,20090608X30506,1,0.013502,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.193548,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12426,20120710X40020,1,0.014951,0.005339,0.095238,0.00,0.233333,0.25,0.783784,0.161290,...,0,0,0,1,0,0,0,1,1,0
12427,20210414102909,1,0.012294,0.005339,0.099576,0.00,0.000000,0.25,0.810811,0.193548,...,0,0,0,1,0,0,0,1,1,0
12428,20120823X95947,1,0.007512,0.002482,0.135266,0.00,0.000000,0.25,0.783784,0.000000,...,0,0,0,1,0,0,0,1,1,0
12429,20080902X01358,1,0.025096,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,1,0


In [3]:
target_column = 'severity_class'
x_train, x_test, y_train, y_test=utils.dataset_traintest_split(df, target_column)

In [4]:
x_train = x_train.drop(columns=['ev_id','damage','ev_highest_injury'])
x_test = x_test.drop(columns=['ev_id','damage','ev_highest_injury'])

In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4, min_samples_split=5) 
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)

In [8]:
a, b, c, d, e = utils.print_metrics(y_test, y_pred)

Accuracy: 0.67
Balanced Accuracy: 0.14
Precision: 0.50
Recall: 0.67
F1 Score: 0.57


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
x = df.drop(columns=['severity_class','ev_id','damage','ev_highest_injury'])
y = df[['severity_class']]
k = 5
fold_size = len(x) // k
for i in range(k):
    x_test2 = x.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([x.iloc[:i * fold_size], x.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [11]:
from sklearn.model_selection import KFold, GridSearchCV
cv = KFold(n_splits = 5, shuffle=False)  
dectree= DecisionTreeClassifier() 
params = {
    'criterion': ['gini', 'entropy'],  
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'max_depth':[None, 10, 20, 30]
}

grid_search = GridSearchCV(
    estimator=dectree,       
    param_grid=params,       
    scoring = 'balanced_accuracy',
    cv=cv,
    verbose=2,              
    n_jobs=4,               
)

grid_search.fit(x, y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(), n_jobs=4,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='balanced_accuracy', verbose=2)

In [13]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10}


In [15]:
from sklearn.metrics import balanced_accuracy_score

dectree = DecisionTreeClassifier(
    criterion='entropy',      
    max_depth=None,        
    min_samples_split=10,   
    min_samples_leaf=2,    
    
)

dectree.fit(x_train2, y_train2)
y_pred2 = dectree.predict(x_test2)

balance = balanced_accuracy_score(y_test2, y_pred2)
print(balance*100)

18.189785660719465


In [17]:
a, b, c, d, e = utils.print_metrics(y_test2, y_pred2)

Accuracy: 0.58
Balanced Accuracy: 0.18
Precision: 0.58
Recall: 0.58
F1 Score: 0.58


In [18]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
dh

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20200819X80407,1,0.007710,0.005339,0.039732,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,1,0,0,1,0
1,20170704X12341,1,0.018236,0.005339,0.411220,0.400000,0.000000,0.25,0.783784,0.322581,...,0,1,0,0,0,0,0,1,1,0
2,20141107X32003,1,0.014458,0.005339,0.089816,0.000000,0.000000,0.25,0.783784,0.290323,...,0,1,0,0,0,0,0,1,1,0
3,20090202X02717,1,0.012294,0.005339,0.093661,0.000000,0.333333,0.25,0.810811,0.290323,...,0,0,0,1,0,1,0,0,1,0
4,20100806X95014,1,0.015434,0.000161,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,20220822105779,1,0.007705,0.005339,0.086957,0.040000,0.056667,0.25,0.783784,0.161290,...,0,1,0,0,0,1,0,0,1,0
3104,20130711X70752,1,0.012294,0.005339,0.027605,0.083333,0.000000,0.25,0.783784,0.387097,...,0,0,1,0,0,1,0,0,1,0
3105,20201026102189,1,0.060355,0.001768,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,1,0,0,0,0,1,0,0,1,0
3106,20160621X92619,1,0.011811,0.004447,0.517993,0.000000,0.000000,0.25,0.783784,0.096774,...,0,0,0,1,0,0,0,1,1,0


In [19]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["severity_class"]

In [21]:
y_h_pred = dectree.predict(x_h)

In [22]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.56
Balanced Accuracy: 0.16
Precision: 0.54
Recall: 0.56
F1 Score: 0.55


In [ ]:
### Y-target is DAMAGE, steps followed:
### 1. training the model, 2. using Cross validation and HPT, 3. getting metrics, 4. checking the model validated model on holdout set

In [23]:
df=pd.read_csv('final_ohc_wto_sde_train.csv')
df

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20080506X00599,1,0.012294,0.005339,0.086957,0.00,0.000000,0.25,0.324324,0.225806,...,0,0,0,1,0,0,0,1,1,0
1,20090701X43709,1,0.016400,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.129032,...,0,1,0,0,0,1,0,0,1,0
2,20080827X01344,2,0.014951,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,0,1
3,20090811X65603,1,0.021714,0.005339,0.142068,0.03,0.000000,0.25,0.783784,0.258065,...,0,0,1,0,0,1,0,0,1,0
4,20090608X30506,1,0.013502,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.193548,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12426,20120710X40020,1,0.014951,0.005339,0.095238,0.00,0.233333,0.25,0.783784,0.161290,...,0,0,0,1,0,0,0,1,1,0
12427,20210414102909,1,0.012294,0.005339,0.099576,0.00,0.000000,0.25,0.810811,0.193548,...,0,0,0,1,0,0,0,1,1,0
12428,20120823X95947,1,0.007512,0.002482,0.135266,0.00,0.000000,0.25,0.783784,0.000000,...,0,0,0,1,0,0,0,1,1,0
12429,20080902X01358,1,0.025096,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,1,0


In [24]:
target_column = 'damage'
x_train, x_test, y_train, y_test = utils.dataset_traintest_split(df, target_column)

In [25]:
x_train = x_train.drop(columns=['ev_id','severity_class','ev_highest_injury'])
x_test = x_test.drop(columns=['ev_id','severity_class','ev_highest_injury'])

In [26]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4, min_samples_split=5) 
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)

In [27]:
a, b, c, d, e = utils.print_metrics(y_test, y_pred)

Accuracy: 0.89
Balanced Accuracy: 0.42
Precision: 0.86
Recall: 0.89
F1 Score: 0.87


In [32]:
x = df.drop(columns=['severity_class','ev_id','damage','ev_highest_injury'])
y = df[['damage']]
k = 5
fold_size = len(x) // k
for i in range(k):
    x_test2 = x.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([x.iloc[:i * fold_size], x.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [33]:
from sklearn.model_selection import KFold, GridSearchCV
cv = KFold(n_splits = 5, shuffle=False)  
dectree= DecisionTreeClassifier() 
params = {
    'criterion': ['gini', 'entropy'],  
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'max_depth':[None, 10, 20, 30]
}

grid_search = GridSearchCV(
    estimator=dectree,       
    param_grid=params,       
    scoring = 'balanced_accuracy',
    cv=cv,
    verbose=2,              
    n_jobs=4,               
)

grid_search.fit(x, y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(), n_jobs=4,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='balanced_accuracy', verbose=2)

In [34]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [35]:
from sklearn.metrics import balanced_accuracy_score

dectree = DecisionTreeClassifier(
    criterion='entropy',      
    max_depth=None,        
    min_samples_split=2,   
    min_samples_leaf=1,    
    
)

dectree.fit(x_train2, y_train2)
y_pred2 = dectree.predict(x_test2)

balance = balanced_accuracy_score(y_test2, y_pred2)
print(balance*100)

45.436270921566106


In [36]:
a, b, c, d, e = utils.print_metrics(y_test2, y_pred2)

Accuracy: 0.85
Balanced Accuracy: 0.45
Precision: 0.86
Recall: 0.85
F1 Score: 0.85


In [37]:
dh=pd.read_csv('final_ohc_wto_sde_holdout.csv')
dh

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20200819X80407,1,0.007710,0.005339,0.039732,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,1,0,0,1,0
1,20170704X12341,1,0.018236,0.005339,0.411220,0.400000,0.000000,0.25,0.783784,0.322581,...,0,1,0,0,0,0,0,1,1,0
2,20141107X32003,1,0.014458,0.005339,0.089816,0.000000,0.000000,0.25,0.783784,0.290323,...,0,1,0,0,0,0,0,1,1,0
3,20090202X02717,1,0.012294,0.005339,0.093661,0.000000,0.333333,0.25,0.810811,0.290323,...,0,0,0,1,0,1,0,0,1,0
4,20100806X95014,1,0.015434,0.000161,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,20220822105779,1,0.007705,0.005339,0.086957,0.040000,0.056667,0.25,0.783784,0.161290,...,0,1,0,0,0,1,0,0,1,0
3104,20130711X70752,1,0.012294,0.005339,0.027605,0.083333,0.000000,0.25,0.783784,0.387097,...,0,0,1,0,0,1,0,0,1,0
3105,20201026102189,1,0.060355,0.001768,0.086957,0.000000,0.000000,0.25,0.783784,0.225806,...,1,0,0,0,0,1,0,0,1,0
3106,20160621X92619,1,0.011811,0.004447,0.517993,0.000000,0.000000,0.25,0.783784,0.096774,...,0,0,0,1,0,0,0,1,1,0


In [41]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["damage"]

In [42]:
y_h_pred = dectree.predict(x_h)

In [43]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.85
Balanced Accuracy: 0.44
Precision: 0.84
Recall: 0.85
F1 Score: 0.84


In [ ]:
### Y-target is EV_HIGHEST_INJURY, steps followed:
### 1. training the model, 2. using Cross validation and HPT, 3. getting metrics, 4. checking the model validated model on holdout set

In [44]:
df=pd.read_csv('final_ohc_wto_sde_train.csv')
df

,ev_id,damage,cert_max_gr_wt,afm_hrs_last_insp,apt_elev,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wind_vel_kts,...,med_certf_BASC,med_certf_CL1,med_certf_CL2,med_certf_CL3,med_certf_SPRT,med_crtf_vldty_WOWL,med_crtf_vldty_WTLS,med_crtf_vldty_WWL,crew_tox_perf_N,crew_tox_perf_Y
0,20080506X00599,1,0.012294,0.005339,0.086957,0.00,0.000000,0.25,0.324324,0.225806,...,0,0,0,1,0,0,0,1,1,0
1,20090701X43709,1,0.016400,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.129032,...,0,1,0,0,0,1,0,0,1,0
2,20080827X01344,2,0.014951,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,0,1
3,20090811X65603,1,0.021714,0.005339,0.142068,0.03,0.000000,0.25,0.783784,0.258065,...,0,0,1,0,0,1,0,0,1,0
4,20090608X30506,1,0.013502,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.193548,...,0,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12426,20120710X40020,1,0.014951,0.005339,0.095238,0.00,0.233333,0.25,0.783784,0.161290,...,0,0,0,1,0,0,0,1,1,0
12427,20210414102909,1,0.012294,0.005339,0.099576,0.00,0.000000,0.25,0.810811,0.193548,...,0,0,0,1,0,0,0,1,1,0
12428,20120823X95947,1,0.007512,0.002482,0.135266,0.00,0.000000,0.25,0.783784,0.000000,...,0,0,0,1,0,0,0,1,1,0
12429,20080902X01358,1,0.025096,0.005339,0.086957,0.00,0.000000,0.25,0.783784,0.225806,...,0,0,1,0,0,1,0,0,1,0


In [45]:
target_column = 'ev_highest_injury'
x_train, x_test, y_train, y_test = utils.dataset_traintest_split(df, target_column)

In [48]:
x_train = x_train.drop(columns=['ev_id','severity_class','damage'], errors='ignore')
x_test = x_test.drop(columns=['ev_id','severity_class','damage'], errors='ignore')

In [49]:
dt = DecisionTreeClassifier(max_depth=4, min_samples_split=5) 
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)

In [50]:
a, b, c, d, e = utils.print_metrics(y_test, y_pred)

Accuracy: 0.77
Balanced Accuracy: 0.34
Precision: 0.65
Recall: 0.77
F1 Score: 0.68


C:\Users\Tanushree\miniconda3\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [51]:
x = df.drop(columns=['severity_class','ev_id','damage','ev_highest_injury'])
y = df[['ev_highest_injury']]
k = 5
fold_size = len(x) // k
for i in range(k):
    x_test2 = x.iloc[i * fold_size : (i + 1) * fold_size]
    y_test2= y.iloc[i * fold_size : (i + 1) * fold_size]
    x_train2 = pd.concat([x.iloc[:i * fold_size], x.iloc[(i + 1) * fold_size:]])
    y_train2 = pd.concat([y.iloc[:i * fold_size], y.iloc[(i + 1) * fold_size:]])

In [52]:
from sklearn.model_selection import KFold, GridSearchCV
cv = KFold(n_splits = 5, shuffle=False)  
dectree= DecisionTreeClassifier() 
params = {
    'criterion': ['gini', 'entropy'],  
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'max_depth':[None, 10, 20, 30]
}

grid_search = GridSearchCV(
    estimator=dectree,       
    param_grid=params,       
    scoring = 'balanced_accuracy',
    cv=cv,
    verbose=2,              
    n_jobs=4,               
)

grid_search.fit(x, y)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(), n_jobs=4,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='balanced_accuracy', verbose=2)

In [53]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 4, 'min_samples_split': 2}


In [54]:
from sklearn.metrics import balanced_accuracy_score

dectree = DecisionTreeClassifier(
    criterion='entropy',      
    max_depth=30,        
    min_samples_split=2,   
    min_samples_leaf=4,    
    
)

dectree.fit(x_train2, y_train2)
y_pred2 = dectree.predict(x_test2)

balance = balanced_accuracy_score(y_test2, y_pred2)
print(balance*100)

39.5562434068762


In [55]:
a, b, c, d, e = utils.print_metrics(y_test2, y_pred2)

Accuracy: 0.68
Balanced Accuracy: 0.40
Precision: 0.68
Recall: 0.68
F1 Score: 0.68


In [56]:
dh= pd.read_csv('final_ohc_wto_sde_holdout.csv')

In [57]:
x_h = dh.drop(columns=["severity_class","ev_id", 'damage','ev_highest_injury']) 
y_h = dh["ev_highest_injury"]

In [58]:
y_h_pred = dectree.predict(x_h)

In [59]:
a, b, c, d, e = utils.print_metrics(y_h, y_h_pred)

Accuracy: 0.68
Balanced Accuracy: 0.40
Precision: 0.67
Recall: 0.68
F1 Score: 0.67
